In [1]:
import os
import sys

PROJECT_ROOT = os.path.join(os.path.abspath(os.path.dirname('./../../')), 'src')

sys.path.append(PROJECT_ROOT)

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [2]:
from data_utils import *
from config import Config
from feature_extraction_utils import GlobalClientFeaturesExtractor
from matplotlib import pyplot as plt
import tqdm

**Load data**

In [3]:
%%time
cfg = Config()
ds = DataSource(cfg['data_source'])
ds.read_data()
ds.validation_split()

Reading groups_desc...
Reading train_x...
Reading test_x...
Reading train_target...
Reading test_target...
Validation split: by clientID
CPU times: user 25.4 s, sys: 5.05 s, total: 30.5 s
Wall time: 19.9 s


In [4]:
cfg['data_source'].keys()

dict_keys(['groups_desc', 'train_x', 'test_x', 'train_target', 'test_target'])

In [5]:
print(ds)

Dataset: groups_desc, shape: (204, 2)
Dataset: train_x, shape: (21295142, 4)
Dataset: test_x, shape: (17667328, 4)
Dataset: train_target, shape: (24145, 2)
Dataset: test_target, shape: (20000, 1)
Dataset: valid_x, shape: (5155435, 4)
Dataset: valid_target, shape: (5855, 2)


**Groups descriptions**

In [6]:
# groups = ds.get_data('groups_desc')
# groups

**Train target**

In [6]:
train_target = ds.get_data('train_target')
print(train_target.shape)
train_target.head()

(24145, 2)


,client_id,bins
0,1046,0
1,34089,2
2,34848,1
3,47076,3
4,10938,2


Train clientID distribution

In [8]:
#train_target.client_id.hist(bins=1000);

In [9]:
#train_target.bins.hist();

**Train transactions**

In [7]:
train_x = ds.get_data('train_x')
train_x.shape

(21295142, 4)

In [11]:
#train_x.head()

In [12]:
#train_x.groupby('client_id').trans_date.nunique().hist(bins=100);
#train_x.groupby('client_id').trans_date.max().max()#.hist(bins=100);
#train_x.groupby('client_id').trans_date.min().min()#.hist(bins=100);
#train_x.groupby('client_id').small_group.nunique().hist(bins=100);
#train_x.groupby('client_id').amount_rur.max().hist(bins=100);

Number of clients which spent huge amounts of money in a single transaction

In [13]:
#train_x[train_x.amount_rur > 30000].client_id.nunique()

Some groups are popular, some are not

In [14]:
#train_x.groupby('small_group').client_id.nunique().hist(bins=100)

# Features playground

**Dummy global features per client**

In [12]:
%%time
extractor = GlobalClientFeaturesExtractor(None)
features = extractor.extract(train_x, 3694)
print(f'# features: {len(features)}')

Dates features extraction...
Money features extraction...
Money features, agg level: month
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
Money features, agg level: quarter
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin
amax
sum
std
mean
median
amin

**Extract features**

In [16]:
params_list = [
    (train_x.query(f'client_id == {client_id}'), client_id)
    for client_id in tqdm.tqdm_notebook(train_target.client_id.values[:100])
]

In [ ]:
from multiprocessing.dummy import Pool as ThreadPool

pool = ThreadPool(4)

# Open the urls in their own threads
# and return the results
results = pool.map(, urls)

#close the pool and wait for the work to finish 
pool.close()
pool.join()

In [13]:
# X_train = np.array([global_extractor.extract(train_x, client_id)
#                     for client_id in tqdm.tqdm_notebook(train_target.client_id.values)])
# y_train = train_target.bins.values

# print(X_train.shape, y_train.shape)

**Global features PCA**

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca = 